In [1]:
import numpy as np
from flask import Flask, request, url_for, redirect, render_template, jsonify, make_response, Blueprint

In [2]:
app = Flask(__name__)

In [3]:
@app.route('/')
def home():
    return render_template('index.html')

In [4]:
@app.route('/dashboard', methods=['GET', 'POST'])
def dashboard():
    if request.method == 'POST':
        # do stuff when the form is submitted

        # redirect to end the POST handling
        # the redirect can be to the same route or somewhere else
        return redirect(url_for('home'))

    # show the form, it wasn't submitted
    return render_template('dashboard.html')

In [5]:
@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        # do stuff when the form is submitted

        # redirect to end the POST handling
        # the redirect can be to the same route or somewhere else
        return redirect(url_for('home'))

    # show the form, it wasn't submitted
    return render_template('predict.html')

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jun/2021 13:20:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2021 13:20:41] "GET /static/fonts/font-awesome-4.7.0/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [16/Jun/2021 13:20:43] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2021 13:20:43] "GET /static/fonts/font-awesome-4.7.0/css/font-awesome.min.css HTTP/1.1" 404 -
